In [0]:
%run "/Users/surajthallapalli@outlook.com/02 Selecting and Renaming Columns in Spark Dataframe/Creating Spark Dataframe 2023-07-11 20:32:19"

+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord| cvandenoor@etsy.com|{+91 8645879087, ...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|      John|        Cena|       john@cena.com|{+91 9886879087, ...| [3, 4]|       true|      900.0|   2022-05-15|2024-03-15 01:16:00|
|  3|     James|        Bond|      james@bond.com|{+91 3245879087, ...|    [2]|      false|      750.6|   2023-01-12|2018-05-05 05:17:02|
|  4|    Robert|      Dowrey|   robert@dowrey.com|                null|     []|       true|        NaN|         null|2019-04-03 08:14:08|
|  5|     Chris|  Hemmsworth|chris

Out[7]: [('id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone_numbers', 'struct<mobile:string,home:string>'),
 ('courses', 'array<bigint>'),
 ('is_customer', 'boolean'),
 ('amount_paid', 'double'),
 ('customer_from', 'date'),
 ('last_updated_ts', 'timestamp')]

In [0]:
users_df['id']

Out[8]: Column<'id'>

In [0]:
from pyspark.sql.functions import col

In [0]:
col('id')

Out[10]: Column<'id'>

In [0]:
type(users_df['id'])

Out[11]: pyspark.sql.column.Column

In [0]:
users_df.select('id', col('first_name'), col('last_name')).show()

+---+----------+------------+
| id|first_name|   last_name|
+---+----------+------------+
|  1|    Corrie|Van den Oord|
|  2|      John|        Cena|
|  3|     James|        Bond|
|  4|    Robert|      Dowrey|
|  5|     Chris|  Hemmsworth|
+---+----------+------------+



In [0]:
users_df.select(users_df['id'], users_df['first_name'],'last_name').show()

+---+----------+------------+
| id|first_name|   last_name|
+---+----------+------------+
|  1|    Corrie|Van den Oord|
|  2|      John|        Cena|
|  3|     James|        Bond|
|  4|    Robert|      Dowrey|
|  5|     Chris|  Hemmsworth|
+---+----------+------------+



In [0]:
# This does not work as there is no object by name u in this session
users_df.alias('u').select(u['id'], col('first_name'), 'last_name').show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3694830399309608>:2
      1 # This does not work as there is no object by name u in this session
----> 2 users_df.alias('u').select(u['id'], col('first_name'), 'last_name').show()

NameError: name 'u' is not defined

In [0]:
# This will work
users_df.alias('u').select('u.id', 'u.first_name', col('last_name')).show()

+---+----------+------------+
| id|first_name|   last_name|
+---+----------+------------+
|  1|    Corrie|Van den Oord|
|  2|      John|        Cena|
|  3|     James|        Bond|
|  4|    Robert|      Dowrey|
|  5|     Chris|  Hemmsworth|
+---+----------+------------+



In [0]:
# This does not work as selectExpr can only take column names or SQL style expressions on column names
users_df.selectExpr(col('id'), 'first_name','last_name').show()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3694830399309610>:2
      1 # This does not work as selectExpr can only take column names or SQL style expressions on column names
----> 2 users_df.selectExpr(col('id'), 'first_name','last_name').show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3063, in DataFrame.selectExpr(self, *expr)
   3061 if len(expr) == 1 and isinstance(expr[0], list):
   3062     expr = expr[0]  # type: ignore[assignment]
-> 3063 jdf = self._jdf.selectExpr(self._jseq(e

In [0]:
from pyspark.sql.functions import concat,lit,col

In [0]:
users_df. \
    select(
        'id','first_name','last_name',
        concat(users_df['first_name'], lit(','),col('last_name')).alias('full_name')
    ).show()

+---+----------+------------+-------------------+
| id|first_name|   last_name|          full_name|
+---+----------+------------+-------------------+
|  1|    Corrie|Van den Oord|Corrie,Van den Oord|
|  2|      John|        Cena|          John,Cena|
|  3|     James|        Bond|         James,Bond|
|  4|    Robert|      Dowrey|      Robert,Dowrey|
|  5|     Chris|  Hemmsworth|   Chris,Hemmsworth|
+---+----------+------------+-------------------+



In [0]:
users_df.alias('u').selectExpr('id','first_name','last_name', "concat(u.first_name, ',', u.last_name) as full_name").show()

+---+----------+------------+-------------------+
| id|first_name|   last_name|          full_name|
+---+----------+------------+-------------------+
|  1|    Corrie|Van den Oord|Corrie,Van den Oord|
|  2|      John|        Cena|          John,Cena|
|  3|     James|        Bond|         James,Bond|
|  4|    Robert|      Dowrey|      Robert,Dowrey|
|  5|     Chris|  Hemmsworth|   Chris,Hemmsworth|
+---+----------+------------+-------------------+



In [0]:
users_df.createOrReplaceTempView('users')

In [0]:
spark.sql("""
          SELECT id,first_name,last_name,
          concat(u.first_name, ',', u.last_name) AS full_name
          FROM users as u
          
 """).show()

+---+----------+------------+-------------------+
| id|first_name|   last_name|          full_name|
+---+----------+------------+-------------------+
|  1|    Corrie|Van den Oord|Corrie,Van den Oord|
|  2|      John|        Cena|          John,Cena|
|  3|     James|        Bond|         James,Bond|
|  4|    Robert|      Dowrey|      Robert,Dowrey|
|  5|     Chris|  Hemmsworth|   Chris,Hemmsworth|
+---+----------+------------+-------------------+

